# AVL Tree

## Implementing a binary search tree first

In [41]:
class BST_Node():
    def __init__(self, value):
        self.value = value
        self.left = None
        self.right = None
        
    # call recursively 
    def add(self, value):
        if value<=self.value:
            if self.left is None:
                self.left = BST_Node(value)
            else:
                self.left.add(value)
        else:
            if self.right is None:
                self.right = BST_Node(value)
            else:
                self.right.add(value)

    def __str__(self, level=0):
        return_value = '  '*level + str(self.value) + '\n'
        left_value = '' if self.left is None else self.left.__str__(level+1)
        right_value = '' if self.right is None else self.right.__str__(level+1)
        return_value = left_value + return_value + right_value
        return return_value
    
    
    # traversal by using queue
    def levelOrder(self):
        queue = Q()
        queue.enQ(self)
        return_list = []
        
        while queue.is_empty() is False:
            # 拿出一個 node
            current_node = queue.deQ()
            return_list.append(str(current_node.value))
            print(f"Node:{current_node.value}, Level:{self.count_level(current_node)}")
            
            # 把兩邊children 加上去
            if current_node.left is not None:
                queue.enQ(current_node.left)
            if current_node.right is not None:
                queue.enQ(current_node.right)
            
        return ' '.join(return_list)
    
    # my solution
    def count_level(self, current_node=None):
        # initial call, set as self
        if current_node is None:
            current_node = self
            
        # right arm, left arm, 取高的, 加上1
        # 從最末端往回推算。leaf node -> return 1 -> 依序往上加
        right = 0 if current_node.right is None else self.count_level(current_node.right)
        left = 0 if current_node.left is None else self.count_level(current_node.left)
        
        return max(left,right)+1
    
b = BST_Node(50)
b.add(40)
b.add(1)
b.add(45)
b.add(20)
b.add(25)
b.add(103)
b.add(143)
print(b)

b.levelOrder()

    1
      20
        25
  40
    45
50
  103
    143

Node:50, Level:5
Node:40, Level:4
Node:103, Level:2
Node:1, Level:3
Node:45, Level:1
Node:143, Level:1
Node:20, Level:2
Node:25, Level:1


'50 40 103 1 45 143 20 25'

In [28]:
class Q():
    def __init__(self):
        self.head = None
        self.tail = None
        
    def enQ(self, value):
        new_node = N(value)
        
        if self.head is None:
            self.head = new_node
            self.tail = new_node
        else:
            self.tail.next = new_node
            self.tail = new_node
            
    def __iter__(self):
        current_node = self.head
        while current_node:
            yield current_node
            current_node = current_node.next
            
    def __str__(self):
        ret_str = [str(n.value) for n in self if n.value is not None]
        return ' '.join(ret_str)
            
    def is_empty(self):
        if self.head is None:
            return True
        else:
            return False
    
    def deQ(self):
        if self.is_empty():
            return None
        else:
            ret_value = self.head.value
            self.head =self.head.next
            return ret_value
        
class N():
    def __init__(self, value):
        self.value = value
        self.next = None
        
